In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [3]:
X=np.load("X.npy")
Y=np.load("Y.npy")

In [4]:
# Expand the dimensions to add a dimension in between ie from N,100 to N,1,100
# This is because conv1D requires this format of N,C,L
# N = mini batch size 
# C = Channels : 1 in this case
# L = length or no of features : 100 here
X=np.expand_dims(X,axis=1)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05, random_state=42)

In [6]:
X.shape

(3263, 1, 100)

In [7]:
input_size = 100
num_classes = 500
batch_size = 100
learning_rate = 0.0001

In [20]:
class Net(nn.Module):
    def __init__(self,input_size,num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv1d(10, 20, kernel_size=5)
        self.conv3 = nn.Conv1d(20, 30, kernel_size=5)
        self.conv4 = nn.Conv1d(30, 40, kernel_size=5)
        self.conv5 = nn.Conv1d(40, 50, kernel_size=5)
        self.conv2_drop = nn.Dropout()
        self.fc1 = nn.Linear(4000, 50)
        self.fc2 = nn.Linear(50, 500)
        self.relu = nn.ReLU()
#         self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.conv2_drop(out)
        
        out = self.conv3(out)
        out = self.relu(out)
        out = self.conv2_drop(out)
        
        out = self.conv4(out)
        out = self.relu(out)
        out = self.conv2_drop(out)
        
        out = self.conv5(out)
        out = self.relu(out)
        out = self.conv2_drop(out)
        
        out = self.relu(out)
        out = out.view(-1,4000)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out
        
#         return out

In [21]:
model = Net(input_size,num_classes)

In [22]:
model.cuda()

Net(
  (conv1): Conv1d (1, 10, kernel_size=(5,), stride=(1,))
  (conv2): Conv1d (10, 20, kernel_size=(5,), stride=(1,))
  (conv3): Conv1d (20, 30, kernel_size=(5,), stride=(1,))
  (conv4): Conv1d (30, 40, kernel_size=(5,), stride=(1,))
  (conv5): Conv1d (40, 50, kernel_size=(5,), stride=(1,))
  (conv2_drop): Dropout(p=0.5)
  (fc1): Linear(in_features=4000, out_features=50)
  (fc2): Linear(in_features=50, out_features=500)
  (relu): ReLU()
)

In [23]:
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [24]:
epochs=100

In [25]:
# test
losses=[]
lr_updates=0
for epoch in range(epochs):
    
    epoch +=1
    for x,y in zip(X_train,Y_train):
        y=np.array([y])
        x=torch.from_numpy(np.expand_dims(x, axis=0))
#         x=torch.from_numpy(x)
        y=torch.from_numpy(y)
        
        inputs = Variable(x.type(torch.cuda.FloatTensor))
        labels = Variable(y.type(torch.cuda.LongTensor))
        
        #clear grads
        optimizer.zero_grad()
#         print("gradients cleared")
        #forward to get predicted values
        
        outputs = model.forward(inputs)
#         print("Forward pass done")
        loss = criterion(outputs, labels)
#         print("loss calculated")
        
        
        loss.backward()# back props
#         print("backprop done")
        optimizer.step()# update the parameters
    losses.append(float(loss))
    print('epoch {}, loss {}'.format(epoch,loss.data[0]))

epoch 1, loss 0.41005468368530273
epoch 2, loss 0.6029276847839355
epoch 3, loss 0.6363272666931152
epoch 4, loss 0.3928227424621582
epoch 5, loss 0.5231156349182129
epoch 6, loss 0.3263363838195801
epoch 7, loss 0.5751805305480957
epoch 8, loss 0.4044961929321289
epoch 9, loss 0.32589149475097656
epoch 10, loss 0.5601582527160645
epoch 11, loss 0.2398052215576172
epoch 12, loss 0.5331926345825195
epoch 13, loss 0.40486717224121094
epoch 14, loss 0.2447977066040039
epoch 15, loss 0.38222599029541016
epoch 16, loss 0.29659223556518555
epoch 17, loss 0.5030755996704102
epoch 18, loss 0.6226072311401367
epoch 19, loss 0.3017759323120117
epoch 20, loss 0.34270477294921875
epoch 21, loss 0.42222023010253906
epoch 22, loss 0.5824832916259766
epoch 23, loss 0.5717935562133789
epoch 24, loss 0.6430435180664062
epoch 25, loss 0.15785884857177734
epoch 26, loss 0.4818587303161621
epoch 27, loss 0.3072361946105957
epoch 28, loss 0.7268223762512207
epoch 29, loss 0.556365966796875
epoch 30, loss 0

In [26]:
outputs.shape

torch.Size([1, 500])

In [27]:
labels.shape

torch.Size([1])

In [28]:
# FOR TRAIn MODEL
model_pred=model(Variable(torch.from_numpy(X_train).type(torch.cuda.FloatTensor)))

In [29]:
correct=0
total_train_compounds=len(Y_train)
for i,compound in enumerate(model_pred):
    
    
    max_,max_index=torch.max(compound,0)
    if (int(max_index)==Y_train[i]):
        correct+=1

print("The train accuracy is {}".format((correct/total_train_compounds)*100))

The train accuracy is 74.86285898676994


In [30]:
# Testing 

In [31]:
# FOR TEST MODEL
model_pred_test=model(Variable(torch.from_numpy(X_test).type(torch.cuda.FloatTensor)))

In [32]:
# The test accuracy
test_correct=0
total_test_compounds=len(Y_test)
Y_test_pred=[]
for i,test_compound in enumerate(model_pred_test):
    
    
    max_,max_index=torch.max(test_compound,0)
    Y_test_pred.append(int(max_index))
    if (int(max_index)==Y_test[i]):
        test_correct+=1

print("The test accuracy is {}".format((test_correct/total_test_compounds)*100))

The test accuracy is 76.21951219512195


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# FOR THE ENTIRE DATASET

In [ ]:
model_pred_total=model(Variable(torch.from_numpy(X).type(torch.cuda.FloatTensor)))

In [ ]:
# The test accuracy
total_correct=0
total_compounds=len(Y)
Y_predicited=[]
for i,compound in enumerate(model_pred_total):

    max_,max_index=torch.max(compound,0)
    Y_predicited.append(int(max_index))
    if (int(max_index)==Y[i]):
        total_correct+=1

print("The total accuracy is {}".format((total_correct/total_compounds)*100))

In [ ]:
plt.plot(Y_predicited)

In [ ]:
torch.save(model.state_dict(),"Classification using DNN-Conv1D.pyt")

In [ ]:
Y

In [ ]:
Y_predicited

In [ ]:
plt.plot(Y_predicited)

In [ ]:
Y.shape

In [ ]:
cor=zip(Y_predicited,Y)

In [ ]:
plt.plot(list(cor))

In [ ]:
list(cor)

In [ ]:
plt.plot(losses)

In [ ]:
type(losses)

In [ ]:
losses

In [ ]:
list(Y)

In [ ]:
np.save("Y_pred_conv1D",Y_predicited)

In [ ]:
%matplotlib notebook

In [ ]:
np.save("Y_pred_conv",Y_test_pred)

In [ ]:
np.save("Y_test",Y_test)